The goal of this notebook is to develop a data preprocessing script that will: 
- Read in a sea ice data file
- determine the type of data
- standardize the naming conventions of (x,y) dimensions as (ni,nj), the data and errors as (data), (error), and the latitude and longitude as (lat, lon)
- write a netcdf file containing the necessary data and metadata to a desired location

In [1]:
import xarray as xr
# import pandas as pd
import os
import glob
import h5py
import numpy as np

In [36]:
icesat2_names = ['ICESAT2', 'ICESAT-2', 'IS2', 'IS-2', 'Icesat2','Icesat-2','IceSat-2', 'icesat2','icesat-2','is2','is-2']


In [37]:
name = 'is2'

In [42]:
def determine_data_source(name, data_type):
    if name in icesat2_names:
        if data_type in ['freeboard', 'fb', 'hi']:
            print("You're converting freeboard data from NASA's ICESat-2 mission.")
        read_is2_data(data_type)
    else:
        raise NotImplementedError

In [61]:
def read_is2_data(data_type, start_date = '2019.01.01', tracks = ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l'], 
                  input_path = '/glade/scratch/mollyw/external_data/ICESat-2/thickness/',
                  output_path = '/glade/scratch/mollyw/external_data/IS2_reduced/thickness/'):
    
    days_of_year = [31,29,31,30,31,30,31,31,30,31,30,31]
    year = start_date[0:4]
    dir_list = [name for name in sorted(os.listdir(input_path))]
    print(dir_list)
    
    if os.path.isdir(output_path) is False:
        print('Output directory presently does not exist! Generating directory now.')
        os.makedirs(output_path)
    else:
        print('Output directory exists! Nice!')
    
    # THICKNESS PREP
    if data_type in ['thickness', 'hi', 'sithick','SIT']:
        for dir_day in dir_list:
            files = sorted(glob.glob(input_path+dir_day+'/IS2SITDAT4_01*.nc'))
            if len(files) == 0:
                FileNotFoundError('I did not find any IS2 thickness data in your system for this date. Please check if data is available in correct directories.')
            else:
                generate_thickness_netcdf(files, dir_day, output_path)
                
        check = os.listdir(output_path)
        if len(check) == 0:
            FileNotFoundError('No files were converted! Please debug.')
    
    # FREEBOARD PREP
    elif data_type in ['freeboard', 'fb', 'FB']:
        for dir_day in dir_list:
            files = sorted(glob.glob(input_path+dir_day+'/ATL10-01*[0-9]*.h5'))
            if len(files) == 0:
                FileNotFoundError('I did not find any IS2 freeboard data in your system for this date. Please check if data is available in correct directories.')
            else:
                generate_freeboard_netcdf(files, tracks, dir_day, output_path)
                
        check = os.listdir(output_path)
        if len(check) == 0:
            FileNotFoundError('No files were converted! Please debug.')
    
    # YET NOT IMPLEMENTED PREP      
    elif data_type in ['concentration', 'siconc','aice','SIC']:
        raise NotImplementedError('Not finished')
        
    elif data_type in ['motion']:
        raise NotImplementedError('Not currently supported')
        
    elif data_type in ['snow']:
        raise NotImplementedError('Not currently supported')
        
    elif data_type in ['ice_age', 'age']:
        raise NotImplementedError('Not currently supported')
        
    else:
        raise NotImplementedError("The type of data you wish to convert is not supported. Please contact DAReS if you wish to add support for your data type.")

In [62]:
determine_data_source(name, 'hi')

You're converting freeboard data from NASA's ICESat-2 mission.
['2019.01.01', '2019.01.02', '2019.01.03', '2019.01.04', '2019.01.05', '2019.01.06', '2019.01.07', '2019.01.08', '2019.01.09', '2019.01.10', '2019.01.11', '2019.01.12', '2019.01.13', '2019.01.14', '2019.01.15', '2019.01.16', '2019.01.17', '2019.01.18', '2019.01.19', '2019.01.20', '2019.01.21', '2019.01.22', '2019.01.23', '2019.01.24', '2019.01.25', '2019.01.26', '2019.01.27', '2019.01.28', '2019.01.29', '2019.01.30', '2019.01.31']
Output directory exists! Nice!


OverflowError: time values outside range of 64 bit signed integers

In [9]:
def generate_freeboard_netcdf(files, tracks, dir_day, output_path):
    data_list = []
    for file in files:
        with h5py.File(file, mode='r') as f:
            for track in tracks:
                if len(f['%s/freeboard_beam_segment/' %track].keys()) < 2:
                    pass
                else:
                    # print(f['/%s/freeboard_beam_segment/latitude' %track])
                    # access the latitude
                    latvar = f['/%s/freeboard_beam_segment/latitude' %track]
                    # and rename, as a long list?
                    latitude = latvar[:]

                    # access the longitude
                    lonvar = f['/%s/freeboard_beam_segment/longitude' %track]
                    # and rename, as a long list?
                    longitude = lonvar[:]

                    # access the freeboard values 
                    dset_name = '/%s/freeboard_beam_segment/beam_fb_height' %track
                    datavar = f[dset_name]
                    # and rename as a long list
                    data = datavar[:]

                    # access the freeboard uncertainty
                    unc_var = f['/%s/freeboard_beam_segment/beam_fb_sigma' %track]
                    unc = unc_var[:]

                    # collect metadata
                    units = datavar.attrs['units']
                    long_name = datavar.attrs['long_name']
                    _FillValue = datavar.attrs['_FillValue']

                    # handle FillValue
                    data[data == _FillValue] = np.nan
                    unc[unc == _FillValue] = np.nan

                    # collect time information
                    timevar = f['/%s/freeboard_beam_segment/delta_time' %track]
                    time = timevar[:]

                    # make dataset, decode time to cftime object, and save in a list 
                    data_list.append(xr.decode_cf(xr.Dataset({"data": (["time"], data),
                                                              "error": (["time"], unc),
                                                              "lon": (["time"], longitude),
                                                              "lat": (["time"], latitude)
                                                             }, coords={"time": (['time'],
                                                                                 time,
                                                                                 {"units": "seconds since 2018-01-01"}
                                                                                )
                                                                       }
                                                            )
                                                 )
                                    )

    day_data = xr.concat(data_list, dim = 'time', compat = 'no_conflicts')
    filename = 'IS2_fb_'+dir_day+'.nc'
    day_data.to_netcdf(output_path+filename)
    print(dir_day + ' done!')

In [60]:
def generate_thickness_netcdf(files, dir_day, output_path):
    data_list = []
    for file in files:
        print
        ds = xr.open_dataset(file)
        
        # access the latitude
        latitude = list(ds.latitude.values)

        # access the longitude
        longitude = list(ds.longitude.values)

        # access the thickness values 
        datavar = ds.ice_thickness
        data = list(datavar.values)

        # access the thickness uncertainty
        unc = list(ds.ice_thickness_unc.values)

        # collect metadata
        units = datavar.attrs['units']
        long_name = datavar.attrs['long_name']

        # collect time information
        time = list(ds.gps_seconds.values)
                    
        # make dataset, decode time to cftime object, and save in a list 
        data_list.append(xr.decode_cf(xr.Dataset({"data": (["time"], data),
                                                  "error": (["time"], unc),
                                                  "lon": (["time"], longitude),
                                                  "lat": (["time"], latitude)
                                                 }, coords={"time": (['time'],
                                                                     time,
                                                                     {"units": "seconds since 1980-01-06"}
                                                                    )
                                                           }
                                                )
                                     )
                        )

    day_data = xr.concat(data_list, dim = 'time', compat = 'no_conflicts')
    filename = 'IS2_hi_'+dir_day+'.nc'
    day_data.to_netcdf(output_path+filename)
    print(dir_day + ' done!')

In [64]:
file = '/glade/scratch/mollyw/external_data/ICESat-2/thickness/2019.01.01/IS2SITDAT4_01_20190101070842_00590201_005_02_bnum1gt1l_001.nc'

In [65]:
ds = xr.open_dataset(file)

In [76]:
file = '/glade/scratch/mollyw/external_data/IS2_reduced/freeboard/IS2_fb_2019.01.01.nc'
ds2 = xr.open_dataset(file)

In [77]:
ds2

<xarray.Dataset>
Dimensions:  (time: 24918)
Coordinates:
  * time     (time) datetime64[ns] 2019-01-01T01:07:21.833454672 ... 2019-01-...
Data variables:
    data     (time) float32 ...
    error    (time) float32 ...
    lon      (time) float64 ...
    lat      (time) float64 ...

In [86]:
xr.decode_cf(ds, decode_timedelta=True)

<xarray.Dataset>
Dimensions:                    (index: 84382)
Coordinates:
  * index                      (index) int64 0 1 2 3 ... 84378 84379 84380 84381
Data variables: (12/19)
    gps_seconds                (index) timedelta64[ns] ...
    height_segment_id          (index) int32 ...
    along_track_distance       (index) float64 ...
    latitude                   (index) float64 ...
    longitude                  (index) float64 ...
    ssh_flag                   (index) int8 ...
    ...                         ...
    ice_thickness_W99mod5dist  (index) float64 ...
    ice_thickness_unc          (index) float64 ...
    ice_thickness_uncsys       (index) float64 ...
    ice_thickness_uncrandom    (index) float64 ...
    ice_type                   (index) int64 ...
    region_flag                (index) uint8 ...
Attributes:
    Description:  Along track sea ice thickness estimates from ICESat-2 and a...
    Contact:      Alek Petty (alek.a.petty@nasa.gov)
    Reference:    Data doi: 10.5067/JTI5YG3S6VAJ. Data described in Petty, A....
    Conventions:  CF-1.8
    featureType:  point
    projection:   Data referenced to WGS84 (EPSG:4326)